Your task is to build a claim frequency model that takes as input the attributes of a single property (state, square footage and building age) and outputs the expected number of claims for that building _per unit of exposure_. We would like you to use 1,000 square feet as your unit of exposure. Therefore, if a building is 2,000 square feet, then it corresponds to 2 units of exposure. If your model outputs 3.23 for this building, it will mean that you predict that, on average, this building will generate 6.46 claims per year. 

As we need to take count rate data thus, we will consider the Poisson regression model:

log(λi)=β0+β1xi

Importing all the three Data files:
1) For importing claim data you need to have pyarrow on the system, please refer the environment file

Need to validate below data as we progress
1) Poisson Response The response variable is a count per unit of time or space, described by a Poisson distribution.
2) Independence The observations must be independent of one another.
3) Mean=Variance By definition, the mean of a Poisson random variable must be equal to its variance.
4) Linearity The log of the mean rate, log( λ), must be a linear function of x

In [258]:
import pandas as pd
import numpy as np


In [259]:
df_policy= pd.read_excel('../policies_10272021.xlsx')
df_property=pd.read_csv('../properties_10272021.csv')
df_claim=pd.read_parquet('../claims_10272021.parquet',engine='pyarrow')

In [260]:
df_policy.head(5)

,pol,start,end
0,11bb0dd0,4Jun.2018,4Jun.2019
1,96a3c554,25Jan2017,25Jan2018
2,35a90ece,26Oct2018,26Oct2019
3,6d034563,3Aug.2017,3Aug.2018
4,c70d089a,28Nov.2017,28Nov.2018


In [261]:
df_policy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pol     1760 non-null   object
 1   start   1760 non-null   object
 2   end     1760 non-null   object
dtypes: object(3)
memory usage: 41.4+ KB


sessing start and end columns have "." char need to be removed for preprocessing and also found it has 29feb for non leave years needs to be replaced with 28

In [262]:
df_policy['start']=[i.replace('.','') for i in df_policy['start']]
df_policy['end']=[i.replace('.','') for i in df_policy['end']]

In [263]:
df_policy['start']=[i.replace('29Feb','28Feb') for i in df_policy['start']]
df_policy['end']=[i.replace('29Feb','28Feb') for i in df_policy['end']]


In [264]:
df_policy['start']= pd.to_datetime(df_policy['start'], errors='coerce', format='%d%b%Y')
df_policy['end']= pd.to_datetime(df_policy['end'], errors='coerce', format='%d%b%Y')

In [265]:
df_policy[df_policy['pol']=='254297a1']

,pol,start,end
864,254297a1,2016-02-28,2017-02-28
997,254297a1,2017-02-28,2018-02-28
1504,254297a1,2015-02-28,2016-02-28


In [266]:
df_policy[df_policy['end'].isnull()]

,pol,start,end


In [267]:
df_policy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   pol     1760 non-null   object        
 1   start   1760 non-null   datetime64[ns]
 2   end     1760 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 41.4+ KB


In [269]:
df_policy.head()

,pol,start,end
0,11bb0dd0,2018-06-04,2019-06-04
1,96a3c554,2017-01-25,2018-01-25
2,35a90ece,2018-10-26,2019-10-26
3,6d034563,2017-08-03,2018-08-03
4,c70d089a,2017-11-28,2018-11-28


Policy dataset has three columns (policy number and start and end date) . It doesn't have any null values 

In [270]:
df_property.head()

,prop_id,pol,state,sqft,age
0,7019,0152f838,OH,10876,67
1,8025,604b4377,AZ,87946,95
2,5766,b7007f67,FL,57978,13
3,6120,bae6a6d7,AZ,89202,53
4,1468,a5916966,AZ,35934,29


In [271]:
df_property.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8413 entries, 0 to 8412
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   prop_id  8413 non-null   int64 
 1   pol      8413 non-null   object
 2   state    8413 non-null   object
 3   sqft     8413 non-null   int64 
 4   age      8413 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 328.8+ KB


Policy dataset has 5 columns (property id, policy number, state, sqrt and age). It doesn't have any null values

In [272]:
df_claim.head()

,pol,property,start_date,amount
0,0152f838,7019,25Oct2018,157866.849557
1,0152f838,7019,25Oct2018,918867.179064
2,0152f838,7019,25Oct2018,128602.395049
3,0152f838,7019,25Oct2018,447153.652892
4,0152f838,7019,25Oct2018,221691.940430


In [273]:
df_claim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62565 entries, 0 to 62564
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pol         62565 non-null  object 
 1   property    62565 non-null  int64  
 2   start_date  62565 non-null  object 
 3   amount      62565 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.9+ MB


df_claim has 4 columns,pol number, property id, start date and amount, no null values

Claim dataset is our main dataset from where we can get the claim details, I will try to add other details from policy and property dataset


1) First join Property dataset to Claim 
2) secondly I will try to join policy dataset to Claim

In [274]:
df_property.columns

Index(['prop_id', 'pol', 'state', 'sqft', 'age'], dtype='object')

In [275]:
df_merge=pd.merge(df_claim, df_property,  how='left', left_on=['property'], right_on = ['prop_id'])

In [276]:
df_merge.head()

,pol_x,property,start_date,amount,prop_id,pol_y,state,sqft,age
0,0152f838,7019,25Oct2018,157866.849557,7019,0152f838,OH,10876,67
1,0152f838,7019,25Oct2018,918867.179064,7019,0152f838,OH,10876,67
2,0152f838,7019,25Oct2018,128602.395049,7019,0152f838,OH,10876,67
3,0152f838,7019,25Oct2018,447153.652892,7019,0152f838,OH,10876,67
4,0152f838,7019,25Oct2018,221691.940430,7019,0152f838,OH,10876,67


In [277]:
df_merge['start_date']= pd.to_datetime(df_merge['start_date'], errors='coerce', format='%d%b%Y')

In [278]:
df_merge=df_merge.drop(['pol_y','prop_id'],axis=1)

In [279]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62565 entries, 0 to 62564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pol_x       62565 non-null  object        
 1   property    62565 non-null  int64         
 2   start_date  15300 non-null  datetime64[ns]
 3   amount      62565 non-null  float64       
 4   state       62565 non-null  object        
 5   sqft        62565 non-null  int64         
 6   age         62565 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 3.8+ MB


In [280]:
df_merge = pd.merge(df_merge, df_policy,  how='left', left_on=['pol_x','start_date'], right_on = ['pol','start'])

In [281]:
df_merge.head()

,pol_x,property,start_date,amount,state,sqft,age,pol,start,end
0,0152f838,7019,2018-10-25,157866.849557,OH,10876,67,0152f838,2018-10-25,2019-10-25
1,0152f838,7019,2018-10-25,918867.179064,OH,10876,67,0152f838,2018-10-25,2019-10-25
2,0152f838,7019,2018-10-25,128602.395049,OH,10876,67,0152f838,2018-10-25,2019-10-25
3,0152f838,7019,2018-10-25,447153.652892,OH,10876,67,0152f838,2018-10-25,2019-10-25
4,0152f838,7019,2018-10-25,221691.940430,OH,10876,67,0152f838,2018-10-25,2019-10-25


In [282]:
df_merge.groupby(['property','start_date','age','state','sqft'])['amount'].count()

property  start_date  age  state  sqft 
46        2017-01-06  34   FL     75772    5
          2018-01-06  34   FL     75772    4
47        2017-01-06  16   AZ     62822    1
          2018-01-06  16   AZ     62822    4
48        2017-01-06  87   AZ     98971    7
                                          ..
8403      2018-01-25  45   FL     19760    3
8404      2017-01-25  75   AZ     6894     4
          2018-01-25  75   AZ     6894     5
8405      2017-01-25  54   AZ     30015    6
          2018-01-25  54   AZ     30015    5
Name: amount, Length: 3522, dtype: int64

In [283]:
df_final=df_merge.groupby(['property','start_date','age','state','sqft'])['amount'].count()
df_final=df_final.reset_index()

In [284]:
df_final.head()

,property,start_date,age,state,sqft,amount
0,46,2017-01-06,34,FL,75772,5
1,46,2018-01-06,34,FL,75772,4
2,47,2017-01-06,16,AZ,62822,1
3,47,2018-01-06,16,AZ,62822,4
4,48,2017-01-06,87,AZ,98971,7


In [285]:
df_final.rename(columns={'amount': 'claims'}, inplace=True)

In [286]:
df_final.head()

,property,start_date,age,state,sqft,claims
0,46,2017-01-06,34,FL,75772,5
1,46,2018-01-06,34,FL,75772,4
2,47,2017-01-06,16,AZ,62822,1
3,47,2018-01-06,16,AZ,62822,4
4,48,2017-01-06,87,AZ,98971,7


In [287]:
df_final['exposure']=df_final['sqft']/1000


In [288]:
df_final['start_date']=pd.to_datetime(df_final['start_date'])

In [289]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   property    3522 non-null   int64         
 1   start_date  3522 non-null   datetime64[ns]
 2   age         3522 non-null   int64         
 3   state       3522 non-null   object        
 4   sqft        3522 non-null   int64         
 5   claims      3522 non-null   int64         
 6   exposure    3522 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 192.7+ KB


In [290]:
df_final['month']=[ i.month for i in df_final['start_date']]

In [291]:
df_final.head()

,property,start_date,age,state,sqft,claims,exposure,month
0,46,2017-01-06,34,FL,75772,5,75.772,1
1,46,2018-01-06,34,FL,75772,4,75.772,1
2,47,2017-01-06,16,AZ,62822,1,62.822,1
3,47,2018-01-06,16,AZ,62822,4,62.822,1
4,48,2017-01-06,87,AZ,98971,7,98.971,1


In [292]:
df_final['month'].value_counts()

5     1302
1     1236
10     984
Name: month, dtype: int64

In [293]:
df_final['age_bins'] = pd.cut(x=df_final['age'], bins=[-np.inf,0, 10, 20, 30, 40, 50,60,70,80,90,100],labels=['0','10','20', '30', '40','50','60','70','80','90','100'])

In [294]:
df_final=df_final.drop(['start_date','property','age'],axis=1)


In [295]:
df_final['freq']=df_final['claims']/df_final['exposure']

In [301]:
print(f"Variance of freq {df_final['freq'].var()}")
print(f"mean of freq {df_final['freq'].mean()}")

Variance of freq 0.09113267896043019
mean of freq 0.15972516824340868


In [303]:
print(f"Variance of claim {df_final['claims'].var()}")
print(f"mean of claim {df_final['claims'].mean()}")

Variance of claim 6.379131877026931
mean of claim 4.34412265758092


In [296]:
df_final.to_csv('Output_files/Data_pre.csv',index=False)

*************************************************************************************************end of the file*****************